In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from time import time, sleep

*Task 5* fill the function definitions based on your solution to part 1 of the list. Note, that some function signatures (arguments and return values) have changed - the state now needs to be passed as an argument and returned.

**Do not modify the templates besides changing the ellipsis (...) into your code**

In [2]:
def initialize(N, E):
    # Calculate the number of agents in each group
    num_agents = int((N * N) * (1 - E) / 2)

    # Randomly assign agents to the first group (1) and second group (2)
    e = N * N - 2 * num_agents
    ones_list = [1] * num_agents
    two_list = [2] * num_agents
    zeros_list = [0] * e
    agents_ = ones_list + two_list + zeros_list
    np.random.shuffle(agents_)
    agents = np.array(agents_).reshape(N, N)
    zeros_matrix = np.zeros((N+2, N+2))
    zeros_matrix[1:-1, 1:-1] = agents

    return zeros_matrix


def mid(a):
    return a[1:-1, 1:-1]


def return_unhappy(state, t, N):
    unhappy_agents = []
    
    # Iterate over each agent
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            agent = state[i, j]
            
            g = agent
            
            if g == 1 or g == 2:
            
                b = state[i-1:i+2, j-1:j+2]
                b = b.copy()
                b[1,1] = 0
                sum_ = np.sum(b==g)

                all_neighbors = np.sum(b!=0)
                
                if all_neighbors != 0:
                    same_group_neighbors = sum_ / all_neighbors
                    if same_group_neighbors < t:
                        unhappy_agents.append((i, j))
    
    return unhappy_agents


def get_vacant(state, N):
    vacant = []
    # Iterate over each cell in the inner part of the state array
    for i in range(1, N+1):
        for j in range(1, N+1):
            if state[i, j] == 0:
                vacant.append((i, j))
    return vacant


def move(state, unhappy, N):
    np.random.shuffle(unhappy)
    
    for agent_pos in unhappy:
        agent_row, agent_col = agent_pos
        
        # Get the vacant positions
        vacant = get_vacant(state, N)
        
        # Randomly select a vacant position
        new_pos = np.random.choice(len(vacant))
        new_row, new_col = vacant[new_pos]
        
        # Move the agent to the vacant position
        state[new_row, new_col] = state[agent_row, agent_col]
        state[agent_row, agent_col] = 0
        
    return state

# Simulate function should:
#  - initialize the array
#  - run the code as long as there are unhappy agents AND we did not hit the iteration limit
#  - return True or False value:
#    - if all agents are happy before the limit of iterations is hit: True
#    - if not - the simulation should stop and False should be returned
def simulate(N, t, E):
    max_iterations=60
    state = initialize(N, E)
    iteration = 0

    while True:
        unhappy_agents = return_unhappy(state, t, N)

        if len(unhappy_agents) == 0:
            # All agents are happy
            return True

        if iteration >= max_iterations:
            # Hit the iteration limit
            return False

        # Move unhappy agents
        state = move(state, unhappy_agents, N)
        iteration += 1

*Task 6* After copying and modifying the implementation from part 1, prepare a simulation that will solve the following problem.

Imagine you are a planning a new development for up to 900 inhabitants (on a grid of 30x30) and you know that two groups will be equally interested in occupying it (the groups will be of equal size). However, you do not have a full information regarding the satisfaction threshold $t$ of the agents (the satisfaction is computed as a fraction of the neighbors of the same group among all of up to 8 of the agent's neighbors) - you only know that it lies somewhere in the range between 0.5 and 0.75.

The agents will be able to move once every month (each month = one iteration), and the requirement is to make all agents satisfied in at least 95% of the simulation runs in at most 5 years (60 iterations).

The goal - to find the minimum proportion of empty space in the system that fulfills the requirements above for values of $t$ in list \[0.5, 0.55, 0.6, 0.65, 0.7, 0.75\]. For each value of $t$, run 20 repetitions of the simulation with each proportion of empty spaces *E* - from the list of values:
\[0.1,
 0.12,
 0.14,
 0.16,
 0.18,
 0.2,
 0.22,
 0.24,
 0.26,
 0.28,
 0.3,
 0.32,
 0.34,
 0.36,
 0.38,
 0.4\]
 and note the minimum value of *E* for each $t$.

In [ ]:
%%time
N = 30
n_iter = 20
min_empty_spaces = {}

for t in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]:
    min_empty_spaces[t] = float('inf')

    for E in [0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4]:
        satisfied_count = 0

        for i in range(n_iter):
            result = simulate(N, t, E)

            if result:
                satisfied_count += 1

        satisfaction_percentage = satisfied_count / n_iter * 100

        if satisfaction_percentage >= 95:
            min_empty_spaces[t] = min(min_empty_spaces[t], E)

print(min_empty_spaces)